# Cover Letter Generator
In this notebook we are going to create a Cover letter generator RAG using LangChain

## Workflow of Cover Letter Generator
1. Input Collection
* User uploads Resume (PDF/DOCX → parsed into text).
* User pastes Job Description (JD).

2. Keyword Extraction (LangChain pipeline #1)
* Use an LLM + PydanticOutputParser to extract:
    - role (e.g., "Data Scientist")
    - seniority (e.g., "Mid-level")
    - must_have (e.g., Python, SQL, Machine Learning)
    - nice_to_have (e.g., Cloud, Docker, NLP)
    - tools (e.g., TensorFlow, PyTorch, Tableau)
✅ Output = structured JDKeywords object.

3. Resume Analysis

* Parse the resume into sections: Experience, Skills, Projects, Education.
* Extract skills & experiences → again with a structured schema (e.g., ResumeProfile).

4. Keyword Matching

* Compare JD keywords vs Resume keywords.
* Mark:
    - ✅ Matches (strengths to emphasize).
    - ❌ Gaps (skills missing → handled carefully, not overclaimed).

5. Cover Letter Generation (LangChain pipeline #2)

* Prompt template uses:
    - JDKeywords (so letter aligns with employer’s needs).
    - ResumeProfile (so letter emphasizes relevant experience).
    - LLM writes a personalized cover letter, structured into:
    - Greeting
    - Hook (why the candidate is interested in this role/company)
    - Body (match candidate’s experience → JD requirements)
    - Closing (enthusiasm + call to action).

configure LLM

In [1]:
from dotenv import load_dotenv
import os
import sys
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

class Settings:
    def __init__(self) -> None:
        sys.path.append(os.path.abspath(".."))
        self.gemini_api = os.environ.get("GOOGLE_API_KEY")
        self.tavily_api_key = os.environ.get("TAVILY_API_KEY")
        
    def load_gemini(self, temp: float = 0.5) -> ChatGoogleGenerativeAI:
        llm = ChatGoogleGenerativeAI(
            model = "gemini-1.5-flash",
            api_key = self.gemini_api,
            temperature = temp
        )
        print("LLM ready:", type(llm).__name__)
        return llm
    def load_gemma(self, temp: float = 0.5)->ChatOpenAI:
        """
        This method returns the local gemma3 model hosted by LM Studio.
        """
        llm = ChatOpenAI(
            model="google/gemma-3-4b",
            openai_api_key = 'lm-studio', # type: ignore
            openai_api_base="http://localhost:1234/v1", # type: ignore
            temperature=temp
        )
        return llm
    
    def load_tavily_search(self, max_results: int = 2) -> TavilySearchResults:
        return TavilySearchResults(max_results = max_results)
        

In [9]:
config = Settings()
llm = config.load_gemini()

LLM ready: ChatGoogleGenerativeAI


## Define the schemas
We will use this schemas to structure our outputs from LLMS

1. schema for extracting keywords

In [5]:
from pydantic import BaseModel, Field
from typing import List, Optional

class JDKeyWords(BaseModel):
    role: Optional[str] = Field(None, description="Role inferred from job description")
    seniority: Optional[str] = Field(None, description="Seniority level like junior, senior, associate etc")
    must_have: List[str] = Field(..., description="Critical must-have skills needed for the job")
    nice_to_have: List[str] = Field(default_factory=list, description="Optional skills")
    tools: List[str] = Field(default_factory=list, description="Software/tools needed for this job")

# ... means the field is required
# Optional[str] means it can be a string or None
# if tools is empty then we will return a []

In [6]:
# cover_letter schema
class CoverLetterOut(BaseModel):
    cover_letter: str = Field(... , description="Generated cover letter text")

## Creating Pipelines

* pipeline to extract keywords from job description

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=JDKeyWords)

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an AI tool that is responsible for extracting hiring signals from job description."
     "Return ONLY valid JSON that matches this schema: \n{format_instructions}"),
     ("human",
      "Job description:\n {job_description}\n"
      "Be precise. Keep lists concise and deduplicated")
]).partial(format_instructions=parser.get_format_instructions())

jd = prompt | llm | parser

In [16]:
job_desc = """
Job Description: Senior Full-Stack Developer
Company: InnovateTech Solutions
Location: San Francisco, CA (Hybrid Remote)
Job Type: Full-Time

About Us
At InnovateTech Solutions, we're building the next generation of SaaS tools that empower businesses to thrive. Our platform leverages cutting-edge AI and data analytics to provide actionable insights. Join our passionate team of engineers and play a key role in shaping our product's future.

The Role
We are seeking a highly skilled and motivated Senior Full-Stack Developer to design, develop, and implement robust software solutions. You will be involved in all stages of the product lifecycle, from concept to deployment, and will mentor junior developers on the team. This is a fantastic opportunity to make a significant impact on a product used by thousands.

Key Responsibilities
Design, code, test, and manage full-stack applications from the database to the UI.

Collaborate with product managers, designers, and other engineers to define, design, and ship new features.

Lead technical architecture discussions and make recommendations on system improvements.

Write clean, maintainable, and efficient code while following best practices.

Conduct code reviews and provide constructive feedback to team members.

Identify and troubleshoot complex performance and scalability issues.

Must-Have Qualifications
5+ years of professional experience in software development.

Frontend: Proven expertise with modern JavaScript frameworks, specifically React and its ecosystem (Redux, Webpack, Hooks).

Backend: Strong proficiency in Python and experience with web frameworks, specifically Django or FastAPI.

Database: Experience with both PostgreSQL and Redis.

Cloud & DevOps: Hands-on experience with AWS (EC2, S3, RDS, Lambda) and familiarity with Docker and CI/CD pipelines.

Solid understanding of RESTful API design principles.

Experience with version control using Git.

Nice-to-Have Qualifications
Experience with TypeScript.

Knowledge of GraphQL.

Familiarity with testing frameworks (e.g., Jest, Pytest, Cypress).

Understanding of agile/scrum development methodologies.

Previous experience in a startup or SaaS environment.

Experience with Kubernetes.

Tools You'll Use
Frontend: React, Redux Toolkit, TypeScript, Vite, Jest

Backend: Python, Django REST Framework, FastAPI, Celery

Database: PostgreSQL, Redis

Infrastructure: AWS, Docker, GitHub Actions, Terraform

Collaboration: Jira, Slack, Figma, Confluence

What We Offer
Competitive salary and equity package.

Comprehensive health, dental, and vision insurance.

401(k) with company matching.

Flexible work schedule and generous PTO.

Professional development budget.

A collaborative, inclusive, and innovative culture.

How to Apply

If you are excited about this opportunity, please apply with your resume and a link to your GitHub profile or portfolio.
"""

In [17]:
def generate_jd_keywords(job_desc: str, llm)->JDKeyWords:

    parser = JsonOutputParser(pydantic_object=JDKeyWords)
    prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an AI tool that is responsible for extracting hiring signals from job description."
     "Return ONLY valid JSON that matches this schema: \n{format_instructions}"),
     ("human",
      "Job description:\n {job_description}\n"
      "Be precise. Keep lists concise and deduplicated")
    ]).partial(format_instructions=parser.get_format_instructions())
    jd = prompt | llm | parser
    result = jd.invoke({"job_description": job_desc})
    
    return result    


In [22]:
import json

llm2 = config.load_gemma()

result = generate_jd_keywords(job_desc , llm2)
print(json.dumps(result , indent=2))

{
  "role": "Senior Full-Stack Developer",
  "seniority": null,
  "must_have": [
    "5+ years of professional experience in software development",
    "Proven expertise with modern JavaScript frameworks, specifically React and its ecosystem (Redux, Webpack, Hooks)",
    "Strong proficiency in Python and experience with web frameworks, specifically Django or FastAPI",
    "Experience with both PostgreSQL and Redis",
    "Hands-on experience with AWS (EC2, S3, RDS, Lambda)",
    "Solid understanding of RESTful API design principles",
    "Experience with version control using Git"
  ],
  "nice_to_have": [
    "Experience with TypeScript",
    "Knowledge of GraphQL",
    "Familiarity with testing frameworks (e.g., Jest, Pytest, Cypress)",
    "Understanding of agile/scrum development methodologies",
    "Previous experience in a startup or SaaS environment",
    "Experience with Kubernetes"
  ],
  "tools": [
    "React",
    "Redux Toolkit",
    "TypeScript",
    "Vite",
    "Jest",
    